<a href="https://colab.research.google.com/github/AndrewstheBuilder/VideoSemanticSearch_JiuJitsu/blob/main/alphapose_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
# 2. Get AlphaPose
!git clone https://github.com/MVIG-SJTU/AlphaPose.git
!cd AlphaPose


Cloning into 'AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 2749 (delta 4), reused 1 (delta 0), pack-reused 2739 (from 1)
Receiving objects: 100% (2749/2749), 118.82 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [3]:
!export PATH=/usr/local/cuda/bin/:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
!pip install cython
!sudo apt-get install libyaml-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libyaml-doc
The following NEW packages will be installed:
  libyaml-dev
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-dev amd64 0.2.2-1build2 [62.8 kB]
Fetched 62.8 kB in 0s (206 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libyaml-dev:amd64.
(Read

In [4]:
%cd /content/AlphaPose/
!python3 setup.py build develop --user

/content/AlphaPose
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Compiling detector/nms/src/soft_nms_cpu.pyx because it changed.
[1/1] Cythonizing detector/nms/src/soft_nms_cpu.pyx
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/trackers
copying trackers/tracker_cfg.py -> build/lib.linux-x86_64-cpython-310/trackers
copying trackers/tracker_api.py -> build/lib.linux-x86_64-cpython-310/

In [5]:
# Install Yolo3 Weights
!pip install gdown
import gdown
# https://drive.usercontent.google.com/download?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC&export=download&authuser=0

file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="yolov3-spp.weights", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
From (redirected): https://drive.google.com/uc?export=download&id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC&confirm=t&uuid=cfa977ac-9269-47cb-b127-5d476c4d54e2
To: /content/AlphaPose/yolov3-spp.weights
100%|██████████| 252M/252M [00:01<00:00, 148MB/s]


'yolov3-spp.weights'

In [6]:
# Move yolov3 weights into appropriate directory
!mkdir ./detector/yolo/data
!mv yolov3-spp.weights ./detector/yolo/data

In [7]:
# Download and move baseline Pose Model into right directory
# https://drive.google.com/file/d/1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw/view
file_id = '1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="simple_res50_256x192.pth", quiet=False)
!mv simple_res50_256x192.pth ./pretrained_models

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw
From (redirected): https://drive.google.com/uc?export=download&id=1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw&confirm=t&uuid=7394a812-9e67-4a88-84e7-1783799f7e6e
To: /content/AlphaPose/simple_res50_256x192.pth
100%|██████████| 136M/136M [00:00<00:00, 141MB/s]


In [8]:
# Download and move Pose tracker model into right directory
# https://drive.google.com/file/d/1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG/view
file_id = '1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth", quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG
To: /content/AlphaPose/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth
100%|██████████| 17.3M/17.3M [00:00<00:00, 81.7MB/s]


'osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth'

In [9]:
!mkdir trackers/weights
!mv osnet.pth ./trackers/weights/

mv: cannot stat 'osnet.pth': No such file or directory


In [10]:
!pip install cython-bbox

  Preparing metadata (setup.py) ... done
  Created wheel for cython-bbox: filename=cython_bbox-0.1.5-cp310-cp310-linux_x86_64.whl size=99103 sha256=b70ca24ff74ffbc82d964991c0300d92d73e82943597242ff1d2c38ed65221f8
  Stored in directory: /root/.cache/pip/wheels/c0/b7/68/bab98b7180cda501101a57fb7d36884218ad45ec60c27cd679
Successfully built cython-bbox


In [11]:
import torch
print(torch.__version__)

2.4.1+cu121


In [12]:
!mkdir /content/AlphaPose/img_dir
!mkdir /content/AlphaPose/output_dir

In [13]:
!ls

alphapose
build
configs
detector
docs
examples
img_dir
LICENSE
model_files
osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth
output_dir
pretrained_models
README.md
scripts
setup.cfg
setup.py
trackers


In [17]:
# alpha pose through webcam
# !ls
cfg_file='configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml'
trained_model='pretrained_models/simple_res50_256x192.pth'
img_directory='img_dir/video_1_frame_0960.jpg'
output_directory='output_dir'
# can't open webcam from google collab.
# !python scripts/demo_inference.py --cfg {cfg_file} --checkpoint {trained_model} --outdir examples/res --vis --webcam 0
!python scripts/demo_inference.py --cfg {cfg_file} --checkpoint {trained_model} --image {img_directory} --outdir {output_directory} --save_img --pose_track



Traceback (most recent call last):
  File "/content/AlphaPose/scripts/demo_inference.py", line 13, in <module>
    from detector.apis import get_detector
ModuleNotFoundError: No module named 'detector'
